In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgi
import pandas as pd

In [ ]:
dataset_name = "threads-math-sx"

H = xgi.load_xgi_data(dataset_name)

In [ ]:
print(H)

In [ ]:
simpleH = H.copy()
simpleH.merge_duplicate_edges()
print(simpleH)

In [ ]:
s = H.edges.size
print(f"The min edge size is {s.min()}")
print(f"The max edge size is {s.max()}")
print(f"The mean edge size is {s.mean()}")

In [ ]:
d = H.nodes.degree
print(f"The min degree is {d.min()}")
print(f"The max degree is {d.max()}")
print(f"The mean degree is {d.mean()}")

In [ ]:
cc = [len(c) for c in xgi.connected_components(H)]
vals, counts = np.unique(cc, return_counts=True)
print(len(cc))
pd.DataFrame({"Component Size": vals, "Number": counts})

In [ ]:
def plot_distributions(
    H,
    node_bins=None,
    edge_bins=None,
    log_x1=True,
    log_y1=True,
    log_x2=True,
    log_y2=True,
):

    if node_bins is None:
        node_bins = int(H.num_nodes / 20)
        node_bins = min(node_bins, 100)

    if edge_bins is None:
        s = H.edges.size
        edge_bins = int(s.max() - s.min() + 1)
        edge_bins = min(edge_bins, 100)

    plt.figure(figsize=(8, 4))
    plt.subplot(121)

    h1 = H.nodes.degree.ashist(bins=node_bins, density=True, log_binning=log_x1)
    h2 = H.edges.size.ashist(bins=edge_bins, density=True, log_binning=log_x2)
    y1 = h1[h1.value > 0].value
    y2 = h2[h2.value > 0].value
    ymin = 0.95 * min(y1.min(), y2.min())
    ymax = 1.1 * max(y1.max(), y2.max())

    if log_x1 and log_y1:
        plt.loglog(h1["bin_center"], h1["value"], "ko", markersize=2)
    elif log_x1 and not log_y1:
        plt.semilogx(h1["bin_center"], h1["value"], "ko", markersize=2)
    elif not log_x1 and log_y1:
        plt.semilogy(h1["bin_center"], h1["value"], "ko", markersize=2)
    elif not log_x1 and not log_y1:
        plt.plot(h1["bin_center"], h1["value"], "ko", markersize=2)

    plt.title("Degree distribution")
    plt.xlabel(r"$k$", fontsize=16)
    plt.ylabel(r"$P(k)$", fontsize=16)
    plt.ylim([ymin, ymax])
    sns.despine()

    plt.subplot(122)

    if log_x2 and log_y2:
        plt.loglog(h2["bin_center"], h2["value"], "ko", markersize=2)
    elif log_x2 and not log_y2:
        plt.semilogx(h2["bin_center"], h2["value"], "ko", markersize=2)
    elif not log_x2 and log_y2:
        plt.semilogy(h2["bin_center"], h2["value"], "ko", markersize=2)
    elif not log_x2 and not log_y2:
        plt.plot(h2["bin_center"], h2["value"], "ko", markersize=2)

    plt.title("Edge size distribution")
    plt.xlabel(r"$s$", fontsize=16)
    plt.ylabel(r"$P(s)$", fontsize=16)
    plt.ylim([ymin, ymax])
    sns.despine()
    plt.tight_layout()
    plt.savefig(f"datasheets/{dataset_name}/stats.png", dpi=300)
    plt.show()

In [ ]:
plot_distributions(H, log_x1=True, log_y1=True, log_x2=True, log_y2=True)